In [1]:
from py5paisa import FivePaisaClient
from py5paisa.order import Order, OrderType, Exchange, Bo_co_order
import time
import pandas as pd
import warnings
from datetime import datetime

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
cred = {
    "APP_NAME": "5P56409084",
    "APP_SOURCE": "9217",
    "USER_ID": "pybeQPEbjju",
    "PASSWORD": "AESDZQGqyAa",
    "USER_KEY": "542GTEZguBbLERUXRZe26rLvfWU2X2KD",
    "ENCRYPTION_KEY": "l8sRTIsjl1JhUrz8X48XQJzKGI4k0gft",
}
client = FivePaisaClient(
    email="vasuappliedai@gmail.com", passwd="Vasudeva@2", dob="19981218", cred=cred
)
client.login()

 11:58:03 | Logged in!!


In [5]:
def get_current_expiry_strikes(symbol, start_strike, end_strike):
    indices = script_df.index[script_df["FullName"].str.contains(
        pat=bank_nifty_symbol + " CE", regex=True) == True].tolist()
    call_df = script_df.iloc[indices]
    call_df = call_df[call_df["StrikeRate"] > start_strike]
    call_df = call_df[call_df["StrikeRate"] < end_strike]
    call_df.to_csv("bank_nifty_call_expiry.csv")
    indices = script_df.index[script_df["FullName"].str.contains(
        pat=bank_nifty_symbol + " PE", regex=True) == True].tolist()
    put_df = script_df.iloc[indices]
    put_df = put_df[put_df["StrikeRate"] > start_strike]
    put_df = put_df[put_df["StrikeRate"] < end_strike]
    put_df.to_csv("bank_nifty_put_expiry.csv")
    return call_df, put_df

def map_prices_to_strike_prices(data_frame, entry_point=None):
    temp_rows = []
    for _, row in data_frame.iterrows():
        script_code = row["Scripcode"]
        try:
            if entry_point is None:
                close_point = client.historical_data("N", "D", script_code, "5m", 
                    given_date, given_date).tail(1)
                temp_rows.append(close_point["Close"].iloc[0])
            else:
                close_points = client.historical_data("N", "D", script_code, "1m", 
                    given_date, given_date)
                if close_points is not None:
                    close_points["Datetime"] = pd.to_datetime(close_points["Datetime"])
                    close_point = close_points[close_points["Datetime"]==datetime.fromisoformat(str(entry_point))]["Close"].iloc[0]
                    temp_rows.append(close_point)
                else: 
                    temp_rows.append(9999)
        except:
            temp_rows.append(9999)
            continue
    data_frame["LTP"] = temp_rows
    return data_frame.sort_values(by=["LTP"])


def adjust_strangle(option_type,desired_price,entry_point):
    call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.9, strikePrice*1.1)
    if option_type == "CE":
        call_option_chain = map_prices_to_strike_prices(call_df,entry_point)
        call_script_code = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Scripcode"].iloc[0]
        call_sell = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["LTP"].iloc[0]
        call_symbol = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Name"].iloc[0]
        print("Sold call call_sell, call_symbol, call_script_code ",call_sell,call_symbol, call_script_code)
    if option_type == "PE":
        put_option_chain = map_prices_to_strike_prices(put_df,entry_point)
        desired_put_df = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)
        put_script_code = desired_put_df["Scripcode"].iloc[0]
        put_sell = desired_put_df["LTP"].iloc[0]
        put_symbol = desired_put_df["Name"].iloc[0]
        print("Sold put sell put_sell, put_symbol, put_script_code ",put_sell,put_symbol,put_script_code)


def start_strangle(entry_time,call_script_code,put_script_code):
    call_df = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)
    call_df["Datetime"] = pd.to_datetime(call_df["Datetime"])
    call_df["Close CE"] = call_df["Close"]
    call_df = call_df[call_df["Datetime"]>datetime.fromisoformat(entry_time)]
    call_df = call_df[["Datetime","Close CE"]]
    put_df = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)
    put_df["Datetime"] = pd.to_datetime(put_df["Datetime"])
    put_df["Close PE"] = put_df["Close"]
    put_df = put_df[put_df["Datetime"]>datetime.fromisoformat(entry_time)]
    put_df = put_df[["Datetime","Close PE"]]
    merged_df = pd.merge(call_df,put_df,on="Datetime")
    for index,row in merged_df.iterrows() : 
        call_point = row["Close CE"]
        put_point = row["Close PE"]
        if(call_point*0.5 > put_point):
            print("PE is half of CE at time, call_price and put_price ",row["Datetime"],call_point,put_point)
            adjust_strangle("PE",call_point*0.85,row["Datetime"])
            return "PE",row["Datetime"]
        elif(call_point < put_point*0.5):        
            print("CE is half of PE at index, call_price and put_price ",row["Datetime"],call_point,put_point)
            adjust_strangle("CE",put_point*0.85,row["Datetime"])
            return "CE",row["Datetime"]

def check_sum_not_greater(listA,listB,total):
    for i in range(len(listA)):
        if listA[i]+listB[i] >=total:
            print("Stoploss for straddle hit Call Price",listA[i], " Put Price ",listB[i])
            return

def start_straddle(entry_time,call_script_code,put_script_code,total_premium):
    call_df = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)
    call_df["Datetime"] = pd.to_datetime(call_df["Datetime"])
    call_df = call_df[call_df["Datetime"]>datetime.fromisoformat(entry_time)]
    put_df = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)
    put_df["Datetime"] = pd.to_datetime(put_df["Datetime"])
    put_df = put_df[put_df["Datetime"]>datetime.fromisoformat(entry_time)]  
    call_points = call_df["Close"].values.tolist()
    put_points = put_df["Close"].values.tolist()
    check_sum_not_greater(call_points,put_points,total_premium*1.1)
    print(len(call_points)," ",len(put_points))
       
def rounded_number(number):
    return (round(strikePrice / 1000, 1) * 1000).astype(int)

In [6]:
# This code should be executed on Thursday or when profit is booked
nifty_script_code = 999920000
banknifty_script_code = 999920005
given_date = datetime.today().strftime("%Y-%m-%d")
script_df = pd.read_csv("scripmaster-csv-format.csv",low_memory=False)
bank_nifty_symbol = "BANKNIFTY 03 Nov 2022"
strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date)["Open"].iloc[0]
strikePrice = rounded_number(strikePrice)
strikePrice

41300

In [11]:
call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.9, strikePrice*1.1)
call_df,put_df

(       Exch ExchType  Scripcode                               Name Series  \
 61624     N        D      49275  BANKNIFTY 03 Nov 2022 CE 38000.00     XX   
 61626     N        D      49277  BANKNIFTY 03 Nov 2022 CE 38100.00     XX   
 61628     N        D      49279  BANKNIFTY 03 Nov 2022 CE 38200.00     XX   
 61630     N        D      49281  BANKNIFTY 03 Nov 2022 CE 38300.00     XX   
 61632     N        D      49283  BANKNIFTY 03 Nov 2022 CE 38400.00     XX   
 ...     ...      ...        ...                                ...    ...   
 183179    N        D      49662  BANKNIFTY 03 Nov 2022 CE 43700.00     XX   
 183181    N        D      49664  BANKNIFTY 03 Nov 2022 CE 43800.00     XX   
 183193    N        D      49684  BANKNIFTY 03 Nov 2022 CE 44100.00     XX   
 183198    N        D      49689  BANKNIFTY 03 Nov 2022 CE 44200.00     XX   
 183205    N        D      49696  BANKNIFTY 03 Nov 2022 CE 44300.00     XX   
 
                      Expiry CpType  StrikeRate WireCat       

In [12]:
call_option_chain = map_prices_to_strike_prices(call_df,given_date+"T09:15:00")
put_option_chain = map_prices_to_strike_prices(put_df,given_date+"T09:15:00")

 17:44:06 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:08 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:08 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:09 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:09 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:09 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:10 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:22 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:27 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:27 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:27 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 17:44:27 | Length mismatch: Expected axis has 0 eleme

In [14]:
desired_call_price = 125
desired_put_price = 125

In [15]:
call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)

,Exch,ExchType,Scripcode,Name,Series,Expiry,CpType,StrikeRate,WireCat,ISIN,FullName,LotSize,AllowedToTrade,QtyLimit,Multiplier,Underlyer,Root,TickSize,LTP
183122,N,D,49555,BANKNIFTY 03 Nov 2022 CE 42400.00,XX,2022-11-03 14:30:00,CE,42400.0,,BANKNIFTY,BANKNIFTY 03 Nov 2022 CE 42400.00,25,Y,1200,1,03 Nov 2022,BANKNIFTY,0.05,107.05


In [16]:
put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)

,Exch,ExchType,Scripcode,Name,Series,Expiry,CpType,StrikeRate,WireCat,ISIN,FullName,LotSize,AllowedToTrade,QtyLimit,Multiplier,Underlyer,Root,TickSize,LTP
61671,N,D,49376,BANKNIFTY 03 Nov 2022 PE 40100.00,XX,2022-11-03 14:30:00,PE,40100.0,,BANKNIFTY,BANKNIFTY 03 Nov 2022 PE 40100.00,25,Y,1200,1,03 Nov 2022,BANKNIFTY,0.05,111.4


In [17]:
call_script_code = call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)["Scripcode"].iloc[0]
put_script_code = put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)["Scripcode"].iloc[0]
call_script_code,put_script_code

(49555, 49376)

In [9]:
call_script_code = 49542
put_script_code = 49442

In [10]:
start_strangle(given_date+"T09:16:00",call_script_code,put_script_code)

In [ ]:
start_straddle(given_date+"T09:15:00",call_script_code,put_script_code,1233) 

In [48]:
def day_close_check(call_script_code,put_script_code):
    timestamp = given_date+"T15:25:00"
    call_df = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)
    call_point = call_df[call_df["Datetime"]==timestamp]["Close"].iloc[0]
    put_df = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)
    put_point = put_df[put_df["Datetime"]==timestamp]["Close"].iloc[0]
    if(call_point < put_point*0.75):
        print("Buy CE at ", call_point)
        print("Buy CE and sell CE near to put strike price")
        adjust_strangle("CE",put_point*0.85,timestamp)
    if(put_point < call_point*0.75):
        print("Buy PE at ", put_point)
        print("selling PE near to call strike price")
        adjust_strangle("PE",call_point*0.85,timestamp)
    
day_close_check(call_script_code,put_script_code)